In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/real_estate_data.csv', sep='\t')
df.head(3)

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.7,16.0,51.0,8,NaN,...,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,...,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,...,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0


**2 колонки, с которыми хочу проводить манипуляции:**

In [30]:
df['total_area'] = df['total_area'].round().astype('int')
df.loc[:,['total_area', 'living_area']].sort_values(by='total_area', ascending=True).head(10)

,total_area,living_area
19904,12,10.0
19558,13,NaN
19807,13,10.0
19546,13,NaN
17961,13,NaN
19642,14,11.0
16949,15,NaN
12040,15,NaN
9412,16,13.0
8886,16,10.0


In [41]:
print(df['total_area'].isna().sum(), df['living_area'].isna().sum()) # есть пропуски в колонке 'living_area'

0 1903


# Вопрос:

**Помоги найти решение проблемы.**<br>
Необходимо заменить пустые значения NaN (столбец "living_area") на среднее значение из ближайших данных (на основе равного или меньшего значения "total_area" для "living_area").<br><br>
**Например:**<br>
В строке со значением "total_area"==15 надо заполнить пропуск NaN. Для этого берем срез "total_area"==15 и вытаскиваем медиану 
"living_area".median().<br>
Если есть значение: перезиписываем NaN этим значением.<br>
Если нет значения: "total_area" уменьшаем на 1 (=14) и снова ищем до тех пор, пока не найдем значение.<br>
**В итоге значение станет из "total_area"==14 равное "living_area"=11.0**


Вот пример кода, который должен работать, но он ругается на "размерность" таблиц. Не знаю как исправить и на что повлиять.

In [42]:
tmp = df.groupby('total_area')['living_area'].median().dropna()

df.loc[df['living_area'].isna(), 'living_area'] = \
pd.merge_asof(df['total_area'].reset_index(), tmp, on='total_area').set_index('index')['living_area']

ValueError: left keys must be sorted

**ValueError:** left keys must be sorted

# Этот код "рабочий", только он работает на "стерильных" данных "одинаковой размерности"

**попросил помочь на stackoverflow**<br>
https://stackoverflow.com/questions/78315775/how-to-fill-empty-nan-data-with-median-values-taken-from-the-list-based-on-other)

**Создал искусственную базу данных data:**

In [38]:
data = pd.DataFrame({'total_area': [12, 13, 13, 13, 13, 14, 15, 15, 16, 16], \
'living_area': [10.0, np.nan, 10.0, np.nan, np.nan, 11.0, np.nan, np.nan, 13.0, 10.0]})
data

,total_area,living_area
0,12,10.0
1,13,NaN
2,13,10.0
3,13,NaN
4,13,NaN
5,14,11.0
6,15,NaN
7,15,NaN
8,16,13.0
9,16,10.0


## Вот что ответили:

First compute the median per group (with groupby.median), get rid of the NaNs with dropna,<br>
and perform a merge_asof to map the missing values. Use the output to fill the NaN by boolean indexing:

In [39]:
tmp = data.groupby('total_area')['living_area'].median().dropna()

data.loc[data['living_area'].isna(), 'living_area'] = pd.merge_asof(
    data['total_area'].reset_index(), tmp, on='total_area'
).set_index('index')['living_area']

data

,total_area,living_area
0,12,10.0
1,13,10.0
2,13,10.0
3,13,10.0
4,13,10.0
5,14,11.0
6,15,11.0
7,15,11.0
8,16,13.0
9,16,10.0
